In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scipy.special as special 
from scipy.optimize import curve_fit
import seaborn as sns
from sklearn.metrics import r2_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [2]:
#Cargamos el dataset
df = pd.read_csv('Datos_Limpios_pais_1.csv')
df.head(5)

,Unnamed: 0,id,listing_url,host_since,host_response_time,host_response_rate,host_acceptance_rate,host_is_superhost,host_verifications,neighbourhood_cleansed,...,review_scores_cleanliness,review_scores_checkin,review_scores_communication,review_scores_location,review_scores_value,calculated_host_listings_count,calculated_host_listings_count_entire_homes,calculated_host_listings_count_private_rooms,calculated_host_listings_count_shared_rooms,reviews_per_month
0,0,97945.0,https://www.airbnb.com/rooms/97945,18/04/2011,a few days or more,0%,6%,f,"['email', 'phone']",Hadern,...,4.81,4.90,4.93,4.78,4.73,2.0,2.0,0.0,0.0,0.80
1,1,114695.0,https://www.airbnb.com/rooms/114695,12/05/2011,within an hour,100%,99%,t,"['email', 'phone']",Berg am Laim,...,4.95,4.97,5.00,4.66,4.61,3.0,1.3,0.0,0.0,0.46
2,2,127383.0,https://www.airbnb.com/rooms/127383,26/05/2011,within an hour,100%,92%,f,"['email', 'phone', 'work_email']",Maxvorstadt,...,4.81,4.98,4.96,4.91,4.81,1.0,1.0,0.0,0.0,0.71
3,3,159634.0,https://www.airbnb.com/rooms/159634,01/07/2011,within an hour,100%,78%,f,"['email', 'phone']",Pasing-Obermenzing,...,4.27,4.78,4.80,4.59,4.44,1.0,1.0,0.0,0.0,0.26
4,4,170154.0,https://www.airbnb.com/rooms/170154,14/04/2010,within a few hours,100%,100%,t,"['email', 'phone']",Sendling-Westpark,...,4.99,4.99,4.98,4.77,4.91,1.0,0.0,1.0,0.0,1.00


In [3]:
#Eliminamos las columnas que son innecesarias  
df = df.drop(['Unnamed: 0'], axis=1)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8021 entries, 0 to 8020
Data columns (total 50 columns):
 #   Column                                        Non-Null Count  Dtype  
---  ------                                        --------------  -----  
 0   id                                            8021 non-null   float64
 1   listing_url                                   8021 non-null   object 
 2   host_since                                    8021 non-null   object 
 3   host_response_time                            8021 non-null   object 
 4   host_response_rate                            8021 non-null   object 
 5   host_acceptance_rate                          8021 non-null   object 
 6   host_is_superhost                             8021 non-null   object 
 7   host_verifications                            8021 non-null   object 
 8   neighbourhood_cleansed                        8021 non-null   object 
 9   property_type                                 8021 non-null   o

In [4]:
# Convertimos las columnas 'host_response_rate' y 'host_acceptance_rate' a tipo float
df['host_response_rate'] = df['host_response_rate'].str.rstrip('%').astype('float') / 100.0
df['host_acceptance_rate'] = df['host_acceptance_rate'].str.rstrip('%').astype('float') / 100.0

In [5]:
df['host_is_superhost'] = df['host_is_superhost'].replace('Sin identificar', 'f')

In [6]:
#Declaramos las variables dependientes e independientes para la regresion lodistica
var_ind = df[['review_scores_cleanliness', 'host_response_rate', 'host_acceptance_rate']]
var_dep = df['host_is_superhost']

In [7]:
#Redifinimos las variables
X = var_ind
y = var_dep

In [8]:
#Dividimos el conjunto de datos en un conjunto de entrenamiento y un conjunto de prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=None)

In [9]:
print(y_test.unique())  # Esto debe mostrar solo ['t' 'f']

['f' 't']


In [10]:
#Se escalan todos los datos
escalar = StandardScaler()

In [11]:
#Para realizar el escalamiento de las variables "X" tanto de entrenamiento como de prueba, se utiliza el método fit_transform
X_train = escalar.fit_transform(X_train)
X_test = escalar.transform(X_test)

In [12]:
#Definimos el algoritmo a utilizar
from sklearn.linear_model import LogisticRegression
algoritmo = LogisticRegression()

In [13]:
#Entrenamos el algoritmo
algoritmo.fit(X_train, y_train)

LogisticRegression()

In [14]:
y_pred = algoritmo.predict(X_test)
y_pred

array(['f', 'f', 'f', ..., 'f', 'f', 'f'], dtype=object)

In [15]:
#Verificamos la matriz de confusión
from sklearn.metrics import confusion_matrix
matriz = confusion_matrix(y_test, y_pred)
print('Matriz de confusión:')
print(matriz)

Matriz de confusión:
[[2047    0]
 [ 360    0]]


In [16]:
#Verificamos la precisión
from sklearn.metrics import precision_score
precision = precision_score(y_test, y_pred, average="binary", pos_label="t")
print('Precisión: del modelo', precision)

Precisión: del modelo 0.0


c:\Python\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [17]:
#calculamos la exactitud del modelo
from sklearn.metrics import accuracy_score
exactitud = accuracy_score(y_test, y_pred)
print('Exactitud del modelo:', exactitud)

Exactitud del modelo: 0.8504362276692978


In [18]:
#Calculamos la sensibilidad del modelo
from sklearn.metrics import recall_score
sensibilidad = recall_score(y_test, y_pred, average="binary", pos_label="f")
print('Sensibilidad del modelo:', sensibilidad)

Sensibilidad del modelo: 1.0


In [19]:
print(df['host_is_superhost'].value_counts())

host_is_superhost
f    6818
t    1203
Name: count, dtype: int64


In [20]:
from sklearn.linear_model import LogisticRegression

# Crear el modelo con balance de clases
model = LogisticRegression(class_weight='balanced')

# Entrenar el modelo normalmente
model.fit(X_train, y_train)


LogisticRegression(class_weight='balanced')

In [21]:
y_pred_prob = model.predict_proba(X_test)[:,1]  # Probabilidad de ser "t"
y_pred_adjusted = ['t' if prob > 0.4 else 'f' for prob in y_pred_prob]  # Umbral más bajo


In [22]:
from sklearn.metrics import precision_score

precision = precision_score(y_test, y_pred_adjusted, pos_label="t")
print('Nueva precisión del modelo:', precision)


Nueva precisión del modelo: 0.21709633649932158


In [23]:
from sklearn.metrics import accuracy_score, recall_score, precision_score

# Calcular precisión, exactitud y sensibilidad con el nuevo ajuste
precision = precision_score(y_test, y_pred_adjusted, pos_label="t")
sensibilidad = recall_score(y_test, y_pred_adjusted, pos_label="t")  # Sensibilidad (recall)
exactitud = accuracy_score(y_test, y_pred_adjusted)  # Exactitud (accuracy)

print(f"Precisión ajustada: {precision}")
print(f"Sensibilidad ajustada: {sensibilidad}")
print(f"Exactitud ajustada: {exactitud}")


Precisión ajustada: 0.21709633649932158
Sensibilidad ajustada: 0.8888888888888888
Exactitud ajustada: 0.503946821769838


In [191]:
#Verificamos la matriz de confusión
from sklearn.metrics import confusion_matrix
matriz = confusion_matrix(y_test, y_pred_adjusted)
print('Matriz de confusión:')
print(matriz)

Matriz de confusión:
[[ 893 1154]
 [  40  320]]


**Segunda**

In [24]:
print(df['host_response_rate'].unique())  # Ver valores únicos

[0.   1.   0.9  0.5  0.4  0.86 0.91 0.67 0.89 0.29 0.2  0.6  0.73 0.83
 0.78 0.8  0.17 0.25 0.3  0.7  0.75 0.94 0.27 0.33 0.77 0.95 0.71 0.98
 0.93 0.92 0.43 0.88 0.96 0.82 0.1  0.97 0.87 0.52 0.99 0.63 0.35 0.64
 0.13 0.47 0.85 0.11 0.14 0.36 0.69 0.57 0.09 0.38 0.81 0.18 0.46 0.22
 0.19 0.08 0.56 0.42 0.65 0.12 0.84 0.15 0.31 0.66 0.79 0.68 0.54 0.44
 0.06 0.03]


In [25]:
# df['host_response_rate_category'] = df['host_response_rate'].apply(lambda x: 'Alta' if x >= 0.90 else 'Baja')
median_value = df['host_response_rate'].median()  # Calcula la mediana
df['host_response_rate_category'] = df['host_response_rate'].apply(
    lambda x: 'Alta' if x >= median_value else 'Baja'
)

# Verifica la nueva proporción
print(df['host_response_rate_category'].value_counts(normalize=True))

host_response_rate_category
Alta    0.667872
Baja    0.332128
Name: proportion, dtype: float64


In [26]:
print(df['host_response_rate_category'].value_counts(normalize=True))  # Ver proporciones

host_response_rate_category
Alta    0.667872
Baja    0.332128
Name: proportion, dtype: float64


In [27]:
#Declaramos las variables dependientes e independientes para la regresion lodistica
var_ind1 = df[['host_acceptance_rate', 'calculated_host_listings_count', 'review_scores_communication']]
var_dep1 = df['host_response_rate_category']

In [28]:
#Redifinimos las variables
X1 = var_ind1
y1 = var_dep1

In [29]:
#Dividimos el conjunto de datos en un conjunto de entrenamiento y un conjunto de prueba
X_train1, X_test1, y_train1, y_test1 = train_test_split(X1, y1, test_size=0.3, random_state=None)

In [30]:
#Se escalan todos los datos
escalar = StandardScaler()

In [31]:
#Para realizar el escalamiento de las variables "X" tanto de entrenamiento como de prueba, se utiliza el método fit_transform
X_train1 = escalar.fit_transform(X_train1)
X_test1 = escalar.transform(X_test1)

In [32]:
#Definimos el algoritmo a utilizar
from sklearn.linear_model import LogisticRegression
algoritmo1 = LogisticRegression()

In [33]:
#Entrenamos el algoritmo
algoritmo1.fit(X_train1, y_train1)

LogisticRegression()

In [34]:
y_pred1 = algoritmo1.predict(X_test1)
y_pred1

array(['Alta', 'Alta', 'Alta', ..., 'Alta', 'Alta', 'Alta'], dtype=object)

In [35]:
#Verificamos la matriz de confusión
from sklearn.metrics import confusion_matrix
matriz1 = confusion_matrix(y_test1, y_pred1)
print('Matriz de confusión:')
print(matriz1)

Matriz de confusión:
[[1425  184]
 [ 521  277]]


In [36]:
#Verificamos la precisión
from sklearn.metrics import precision_score
precision1 = precision_score(y_test1, y_pred1, average="binary", pos_label="Alta")
print('Precisión: del modelo', precision1)

Precisión: del modelo 0.7322713257965057


In [37]:
#calculamos la exactitud del modelo
from sklearn.metrics import accuracy_score
exactitud1 = accuracy_score(y_test1, y_pred1)
print('Exactitud del modelo:', exactitud1)

Exactitud del modelo: 0.7071042791857084


In [38]:
#Calculamos la sensibilidad del modelo
from sklearn.metrics import recall_score
sensibilidad1 = recall_score(y_test1, y_pred1, average="binary", pos_label="Alta")
print('Sensibilidad del modelo:', sensibilidad1)

Sensibilidad del modelo: 0.8856432566811684


**Tercera**

In [39]:
print(df['host_acceptance_rate'].unique())  # Ver valores únicos

[0.06 0.99 0.92 0.78 1.   0.67 0.   0.77 0.87 0.98 0.63 0.71 0.96 0.15
 0.97 0.85 0.69 0.82 0.74 0.52 0.83 0.22 0.9  0.5  0.75 0.58 0.72 0.8
 0.73 0.81 0.76 0.47 0.93 0.41 0.38 0.65 0.14 0.94 0.64 0.88 0.42 0.86
 0.4  0.79 0.11 0.25 0.7  0.39 0.2  0.33 0.46 0.43 0.91 0.48 0.03 0.08
 0.44 0.66 0.89 0.6  0.37 0.29 0.53 0.18 0.84 0.54 0.34 0.68 0.1  0.28
 0.51 0.3  0.24 0.04 0.07 0.36 0.57 0.95 0.61 0.27 0.13 0.21 0.59 0.55
 0.09 0.17 0.49 0.12 0.35 0.05 0.26 0.32 0.31 0.23 0.62 0.45 0.16 0.19
 0.56 0.02]


In [40]:
df['host_acceptance_rate_binary'] = df['host_acceptance_rate'].apply(lambda x: 1 if x >= 0.8 else 0)

In [41]:
print(df['host_acceptance_rate_binary'].value_counts(normalize=True))

host_acceptance_rate_binary
1    0.524249
0    0.475751
Name: proportion, dtype: float64


In [42]:
#Declaramos las variables dependientes e independientes para la regresion lodistica
var_ind2 = df[['host_response_rate', 'calculated_host_listings_count', 'review_scores_communication']]
var_dep2 = df['host_acceptance_rate_binary']

In [43]:
#Redifinimos las variables
X2 = var_ind2
y2 = var_dep2

In [44]:
#Dividimos el conjunto de datos en un conjunto de entrenamiento y un conjunto de prueba
X_train2, X_test2, y_train2, y_test2 = train_test_split(X2, y2, test_size=0.3, random_state=None)

In [45]:
#Se escalan todos los datos
escalar = StandardScaler()

In [46]:
#Para realizar el escalamiento de las variables "X" tanto de entrenamiento como de prueba, se utiliza el método fit_transform
X_train2 = escalar.fit_transform(X_train2)
X_test2 = escalar.transform(X_test2)

In [47]:
#Definimos el algoritmo a utilizar
from sklearn.linear_model import LogisticRegression
algoritmo2 = LogisticRegression()

In [48]:
#Entrenamos el algoritmo
algoritmo2.fit(X_train2, y_train2)

LogisticRegression()

In [49]:
y_pred2 = algoritmo2.predict(X_test2)
y_pred2

array([1, 1, 1, ..., 1, 1, 1], dtype=int64)

In [50]:
#Verificamos la matriz de confusión
from sklearn.metrics import confusion_matrix
matriz2 = confusion_matrix(y_test2, y_pred2)
print('Matriz de confusión:')
print(matriz2)

Matriz de confusión:
[[ 426  716]
 [ 123 1142]]


In [51]:
#Verificamos la precisión
from sklearn.metrics import precision_score
precision2 = precision_score(y_test2, y_pred2, average="binary", pos_label=1)
print('Precisión: del modelo', precision2)

Precisión: del modelo 0.6146393972012917


In [52]:
#calculamos la exactitud del modelo
from sklearn.metrics import accuracy_score
exactitud2 = accuracy_score(y_test2, y_pred2)
print('Exactitud del modelo:', exactitud2)

Exactitud del modelo: 0.6514333194848358


In [53]:
#Calculamos la sensibilidad del modelo
from sklearn.metrics import recall_score
sensibilidad2 = recall_score(y_test2, y_pred2, average="binary", pos_label=1)
print('Sensibilidad del modelo:', sensibilidad2)

Sensibilidad del modelo: 0.9027667984189723


In [54]:
import numpy as np

print(np.unique(y_test2))  # Funciona para Pandas y NumPy
print(np.unique(y_pred2))  # Para verificar valores únicos en y_pred2


[0 1]
[0 1]


In [55]:
from sklearn.metrics import confusion_matrix

cm = confusion_matrix(y_test2, y_pred2)
print(cm)

[[ 426  716]
 [ 123 1142]]


**VAR 4**

In [129]:
print(df['property_type'].unique())  # Ver valores únicos

[11 27 30 48 50  9 23 13 31 36  4  7 18 29 19 34 37  0 12  6  8 25 49  1
  5 15 39 44 38 51 40 53 28 42 41 57 16 47 22 10 32 17 14  3 35 26 45 20
 46  2 43 54 24 21 55 56 52 33]


In [140]:
print(df['room_type'].unique())  # Ver valores únicos

['Entire home/apt' 'Private room' 'Shared room' 'Hotel room']


In [141]:
# Mapeo manual de las categorías
mapping = {
    'Entire home/apt': 1,
    'Private room': 2,
    'Shared room': 3,
    'Hotel room': 4
}

# Aplicar el mapeo
df['room_type_encoded'] = df['room_type'].map(mapping)

print(df[['room_type', 'room_type_encoded']])

            room_type  room_type_encoded
0     Entire home/apt                  1
1     Entire home/apt                  1
2     Entire home/apt                  1
3     Entire home/apt                  1
4        Private room                  2
...               ...                ...
8016  Entire home/apt                  1
8017     Private room                  2
8018  Entire home/apt                  1
8019     Private room                  2
8020     Private room                  2

[8021 rows x 2 columns]


In [138]:
df['property_type_binary'] = df['property_type'].apply(lambda x: 1 if x < 15 else 0)

In [ ]:
print(df['property_type_binary'].value_counts(normalize=True))

property_type_binary
1    0.660267
0    0.339733
Name: proportion, dtype: float64


In [142]:
#Declaramos las variables dependientes e independientes para la regresion lodistica
var_ind4 = df[['room_type_encoded', 'review_scores_cleanliness', 'review_scores_communication']]
var_dep4 = df['property_type_binary']

In [143]:
#Redifinimos las variables
X4 = var_ind4
y4 = var_dep4

In [144]:
#Dividimos el conjunto de datos en un conjunto de entrenamiento y un conjunto de prueba
X_train4, X_test4, y_train4, y_test4 = train_test_split(X4, y4, test_size=0.3, random_state=None)

In [145]:
#Se escalan todos los datos
escalar = StandardScaler()

In [146]:
#Para realizar el escalamiento de las variables "X" tanto de entrenamiento como de prueba, se utiliza el método fit_transform
X_train4 = escalar.fit_transform(X_train4)
X_test4 = escalar.transform(X_test4)

In [147]:
#Definimos el algoritmo a utilizar
from sklearn.linear_model import LogisticRegression
algoritmo4 = LogisticRegression()

In [148]:
#Entrenamos el algoritmo
algoritmo4.fit(X_train4, y_train4)

LogisticRegression()

In [149]:
#Realizamos la prediccion
y_pred4 = algoritmo4.predict(X_test4)
y_pred4

array([0, 0, 1, ..., 1, 0, 1], dtype=int64)

In [150]:
#Verificamos la matriz de confusión
from sklearn.metrics import confusion_matrix
matriz4 = confusion_matrix(y_test4, y_pred4)
print('Matriz de confusión:')
print(matriz4)

Matriz de confusión:
[[ 806    6]
 [   0 1595]]


In [151]:
#Verificamos la precisión
from sklearn.metrics import precision_score
precision4 = precision_score(y_test4, y_pred4, average="binary", pos_label=1)
print('Precisión: del modelo', precision4)

Precisión: del modelo 0.9962523422860712


In [152]:
#calculamos la exactitud del modelo
from sklearn.metrics import accuracy_score
exactitud4 = accuracy_score(y_test4, y_pred4)
print('Exactitud del modelo:', exactitud4)

Exactitud del modelo: 0.997507270461155


In [153]:
#Calculamos la sensibilidad del modelo
from sklearn.metrics import recall_score
sensibilidad4 = recall_score(y_test4, y_pred4, average="binary", pos_label=1)
print('Sensibilidad del modelo:', sensibilidad4)

Sensibilidad del modelo: 1.0


**VAR 5**

In [154]:
print(df['review_scores_cleanliness'].unique())  # Ver valores únicos

[4.81 4.95 4.27 4.99 4.83 4.91 4.8  4.76 4.66 4.86 4.67 4.98 4.71 4.75
 4.69 4.96 4.6  4.58 4.94 4.82 4.47 4.87 4.85 4.4  4.88 4.74 4.93 4.19
 4.26 4.21 4.56 4.72 4.57 4.79 4.53 4.5  4.44 4.55 5.   4.73 4.9  4.52
 4.89 4.51 4.59 4.29 4.64 4.92 4.65 4.22 4.41 4.84 4.97 4.25 4.77 4.78
 4.35 4.63 4.54 4.62 4.2  4.42 4.13 4.23 4.38 4.45 4.43 4.33 4.61 4.7
 4.37 4.24 4.68 4.34 4.28 4.48 4.39 4.49 4.32 4.31 4.46 4.3  4.18 4.15
 4.17 4.14 4.36]


In [ ]:
umbral = df['review_scores_cleanliness'].median()  # Calcula la mediana
df['review_scores_cleanliness_binary'] = df['review_scores_cleanliness'].apply(lambda x: 1 if x >= umbral else 0)

review_scores_cleanliness_binary


In [156]:
print(df['review_scores_cleanliness_binary'].value_counts(normalize=True))

review_scores_cleanliness_binary
1    0.507044
0    0.492956
Name: proportion, dtype: float64


In [157]:
#Declaramos las variables dependientes e independientes para la regresion lodistica
var_ind5 = df[['property_type_binary', 'room_type_encoded', 'number_of_reviews']]
var_dep5 = df['review_scores_cleanliness_binary']

In [158]:
#Redifinimos las variables
X5 = var_ind5
y5 = var_dep5

In [159]:
#Dividimos el conjunto de datos en un conjunto de entrenamiento y un conjunto de prueba
X_train5, X_test5, y_train5, y_test5 = train_test_split(X5, y5, test_size=0.3, random_state=None)

In [160]:
#Se escalan todos los datos
escalar = StandardScaler()

In [161]:
#Para realizar el escalamiento de las variables "X" tanto de entrenamiento como de prueba, se utiliza el método fit_transform
X_train5 = escalar.fit_transform(X_train5)
X_test5 = escalar.transform(X_test5)

In [162]:
#Definimos el algoritmo a utilizar
from sklearn.linear_model import LogisticRegression
algoritmo5 = LogisticRegression()

In [163]:
#Entrenamos el algoritmo
algoritmo5.fit(X_train5, y_train5)

LogisticRegression()

In [164]:
#Realizamos la prediccion
y_pred5 = algoritmo5.predict(X_test5)
y_pred5

array([0, 1, 1, ..., 1, 0, 1], dtype=int64)

In [165]:
#Verificamos la matriz de confusión
from sklearn.metrics import confusion_matrix
matriz5 = confusion_matrix(y_test5, y_pred5)
print('Matriz de confusión:')
print(matriz5)

Matriz de confusión:
[[569 579]
 [383 876]]


In [166]:
#Verificamos la precisión
from sklearn.metrics import precision_score
precision5 = precision_score(y_test5, y_pred5, average="binary", pos_label=1)
print('Precisión: del modelo', precision5)

Precisión: del modelo 0.6020618556701031


In [167]:
#calculamos la exactitud del modelo
from sklearn.metrics import accuracy_score
exactitud5 = accuracy_score(y_test5, y_pred5)
print('Exactitud del modelo:', exactitud5)

Exactitud del modelo: 0.6003323639385126


In [168]:
#Calculamos la sensibilidad del modelo
from sklearn.metrics import recall_score
sensibilidad5 = recall_score(y_test5, y_pred5, average="binary", pos_label=1)
print('Sensibilidad del modelo:', sensibilidad5)

Sensibilidad del modelo: 0.6957903097696585


**VAR 6**

In [169]:
print(df['review_scores_communication'].unique())  # Ver valores únicos

[4.93 5.   4.96 4.8  4.98 4.92 4.88 4.9  4.85 4.89 4.79 4.95 4.81 4.97
 4.86 4.84 4.82 4.77 4.94 4.83 4.99 4.87 4.91 4.78 4.76]


In [176]:
print(df['host_verifications'].unique())  # Ver valores únicos

["['email', 'phone']" "['email', 'phone', 'work_email']" "['phone']"
 "['email']" "['phone', 'work_email']" '[]']


In [177]:
df['host_verifications_count'] = df['host_verifications'].apply(lambda x: len(eval(x)))

In [178]:
print(df['host_verifications_count'].value_counts(normalize=True))

host_verifications_count
2    0.784441
1    0.112829
3    0.102606
0    0.000125
Name: proportion, dtype: float64


In [174]:
umbral = df['review_scores_communication'].median()  # Calcula la mediana
df['review_scores_communication_binary'] = df['review_scores_communication'].apply(lambda x: 1 if x >= umbral else 0)

In [175]:
print(df['review_scores_communication_binary'].value_counts(normalize=True))

review_scores_communication_binary
1    0.63446
0    0.36554
Name: proportion, dtype: float64


In [179]:
#Declaramos las variables dependientes e independientes para la regresion lodistica
var_ind6 = df[['number_of_reviews', 'reviews_per_month', 'host_verifications_count']]
var_dep6 = df['review_scores_communication_binary']

In [180]:
#Redifinimos las variables
X6 = var_ind6
y6 = var_dep6

In [181]:
#Dividimos el conjunto de datos en un conjunto de entrenamiento y un conjunto de prueba
X_train6, X_test6, y_train6, y_test6 = train_test_split(X6, y6, test_size=0.3, random_state=None)

In [182]:
#Se escalan todos los datos
escalar = StandardScaler()

In [183]:
#Para realizar el escalamiento de las variables "X" tanto de entrenamiento como de prueba, se utiliza el método fit_transform
X_train6 = escalar.fit_transform(X_train6)
X_test6 = escalar.transform(X_test6)

In [184]:
#Definimos el algoritmo a utilizar
from sklearn.linear_model import LogisticRegression
algoritmo6 = LogisticRegression()

In [185]:
#Entrenamos el algoritmo
algoritmo6.fit(X_train6, y_train6)

LogisticRegression()

In [186]:
#Realizamos la prediccion
y_pred6 = algoritmo6.predict(X_test6)
y_pred6

array([1, 1, 0, ..., 1, 1, 1], dtype=int64)

In [187]:
#Verificamos la matriz de confusión
from sklearn.metrics import confusion_matrix
matriz6 = confusion_matrix(y_test6, y_pred6)
print('Matriz de confusión:')
print(matriz6)

Matriz de confusión:
[[ 580  328]
 [ 177 1322]]


In [188]:
#Verificamos la precisión
from sklearn.metrics import precision_score
precision6 = precision_score(y_test6, y_pred6, average="binary", pos_label=1)
print('Precisión: del modelo', precision6)

Precisión: del modelo 0.8012121212121213


In [189]:
#calculamos la exactitud del modelo
from sklearn.metrics import accuracy_score
exactitud6 = accuracy_score(y_test6, y_pred6)
print('Exactitud del modelo:', exactitud6)

Exactitud del modelo: 0.7901952638138762


In [190]:
#Calculamos la sensibilidad del modelo
from sklearn.metrics import recall_score
sensibilidad6 = recall_score(y_test6, y_pred6, average="binary", pos_label=1)
print('Sensibilidad del modelo:', sensibilidad6)

Sensibilidad del modelo: 0.8819212808539026


**Var 7**

In [126]:
#Verificamos los datos unicos de la columna 'class'
unico = np.unique(df['calculated_host_listings_count'])
unico

array([0., 1.])

In [127]:
#Convierto la variable a dicotomica
df['calculated_host_listings_count'] = df['calculated_host_listings_count'].replace([2, 2.1, 3], 0)
df['calculated_host_listings_count'].head(10)

0    0.0
1    0.0
2    1.0
3    1.0
4    1.0
5    1.0
6    1.0
7    1.0
8    1.0
9    1.0
Name: calculated_host_listings_count, dtype: float64

In [ ]:
#Reemplazar los valores de true por 1 y false a 0 posteriormente lo verificamos 
df['host_is_superhost'] = df['host_is_superhost'].map({'t': True, 'f': False}).astype(int)

In [ ]:
#Declaramos las variables dependientes e independientes para la regresion lodistica
var_ind7 = df[['host_is_superhost', 'host_response_rate', 'host_acceptance_rate']]
var_dep7 = df[['calculated_host_listings_count']]

In [ ]:
#Redifinimos las variables
X7 = var_ind7
y7 = var_dep7

In [ ]:
#Dividimos el conjunto de datos en un conjunto de entrenamiento y un conjunto de prueba
X_train7, X_test7, y_train7, y_test7 = train_test_split(X7, y7, test_size=0.3, random_state=None)

In [ ]:
#Se escalan todos los datos
escalar = StandardScaler()

In [ ]:
#Para realizar el escalamiento de las variables "X" tanto de entrenamiento como de prueba, se utiliza el método fit_transform
X_train7 = escalar.fit_transform(X_train7)
X_test7 = escalar.transform(X_test7)

In [ ]:
#Definimos el algoritmo a utilizar
from sklearn.linear_model import LogisticRegression
algoritmo7 = LogisticRegression()

In [ ]:
#Entrenamos el algoritmo
algoritmo7.fit(X_train7, y_train7)

c:\Python\Lib\site-packages\sklearn\utils\validation.py:1339: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


LogisticRegression()

In [ ]:
#Realizamos la prediccion
y_pred7 = algoritmo7.predict(X_test7)
y_pred7

array([1., 1., 1., ..., 1., 1., 1.])

In [ ]:
#Verificamos la matriz de confusión
from sklearn.metrics import confusion_matrix
matriz7 = confusion_matrix(y_test7, y_pred7)
print('Matriz de confusión:')
print(matriz7)

Matriz de confusión:
[[  88  618]
 [ 134 1567]]


In [ ]:
#Verificamos la precisión
from sklearn.metrics import precision_score
precision7 = precision_score(y_test7, y_pred7, average="binary", pos_label=1)
print('Precisión: del modelo', precision7)

Precisión: del modelo 0.717162471395881


In [ ]:
#calculamos la exactitud del modelo
from sklearn.metrics import accuracy_score
exactitud7 = accuracy_score(y_test7, y_pred7)
print('Exactitud del modelo:', exactitud7)

Exactitud del modelo: 0.6875778977980889


In [ ]:
#Calculamos la sensibilidad del modelo
from sklearn.metrics import recall_score
sensibilidad7 = recall_score(y_test7, y_pred7, average="binary", pos_label=1)
print('Sensibilidad del modelo:', sensibilidad7)

Sensibilidad del modelo: 0.921222810111699


**Var 8**

In [ ]:
#Verificamos los datos unicos de la columna
unico = np.unique(df['reviews_per_month'])
unico

array([0.01, 0.02, 0.03, 0.04, 0.05, 0.06, 0.07, 0.08, 0.09, 0.1 , 0.11,
       0.12, 0.13, 0.14, 0.15, 0.16, 0.17, 0.18, 0.19, 0.2 , 0.21, 0.22,
       0.23, 0.24, 0.25, 0.26, 0.27, 0.28, 0.29, 0.3 , 0.31, 0.32, 0.33,
       0.34, 0.35, 0.36, 0.37, 0.38, 0.39, 0.4 , 0.41, 0.42, 0.43, 0.44,
       0.45, 0.46, 0.47, 0.48, 0.49, 0.5 , 0.51, 0.52, 0.53, 0.54, 0.55,
       0.56, 0.57, 0.58, 0.59, 0.6 , 0.61, 0.62, 0.63, 0.64, 0.65, 0.66,
       0.67, 0.68, 0.69, 0.7 , 0.71, 0.72, 0.73, 0.74, 0.75, 0.76, 0.77,
       0.78, 0.79, 0.8 , 0.81, 0.82, 0.83, 0.84, 0.85, 0.86, 0.87, 0.88,
       0.89, 0.9 , 0.91, 0.92, 0.93, 0.94, 0.95, 0.96, 0.97, 0.98, 0.99,
       1.  , 1.01, 1.02, 1.03, 1.04, 1.05, 1.06, 1.07, 1.08, 1.09, 1.1 ,
       1.11, 1.12, 1.13, 1.14, 1.15, 1.16, 1.17, 1.18, 1.19, 1.2 , 1.21,
       1.22, 1.23, 1.24, 1.25, 1.26, 1.27, 1.28, 1.29, 1.3 , 1.31, 1.32,
       1.33, 1.34, 1.35, 1.36, 1.37, 1.38, 1.39, 1.4 , 1.41, 1.42, 1.43,
       1.44, 1.45, 1.46, 1.47, 1.48, 1.49, 1.5 , 1.

In [ ]:
df['reviews_per_month_binary'] = df['reviews_per_month'].apply(lambda x: 1 if x >= 1 else 0)

In [ ]:
#Declaramos las variables dependientes e independientes para la regresion logistica
var_ind8 = df[['host_is_superhost', 'host_response_rate', 'host_acceptance_rate']]
var_dep8 = df[['reviews_per_month_binary']]

In [ ]:
#Redefinimos las variables
X8 = var_ind8
y8 = var_dep8

In [ ]:
#Dividimos el conjunto de datos en un conjunto de entrenamiento y un conjunto de prueba
X_train8, X_test8, y_train8, y_test8 = train_test_split(X8, y8, test_size=0.3, random_state=None)

In [ ]:
#Se escalan todos los datos
escalar = StandardScaler()

In [ ]:
#Para realizar el escalamiento de las variables "X" tanto de entrenamiento como de prueba, se utiliza el método fit_transform
X_train8 = escalar.fit_transform(X_train8)
X_test8 = escalar.transform(X_test8)

In [ ]:
#Definimos el algoritmo a utilizar
from sklearn.linear_model import LogisticRegression
algoritmo8 = LogisticRegression()

In [ ]:
#Entrenamos el algoritmo
algoritmo8.fit(X_train8, y_train8)

c:\Python\Lib\site-packages\sklearn\utils\validation.py:1339: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


LogisticRegression()

In [ ]:
#Realizamos la prediccion
y_pred8 = algoritmo8.predict(X_test8)
y_pred8

array([0, 0, 1, ..., 0, 0, 0], dtype=int64)

In [ ]:
#Verificamos la matriz de confusión
from sklearn.metrics import confusion_matrix
matriz8 = confusion_matrix(y_test8, y_pred8)
print('Matriz de confusión:')
print(matriz8)

Matriz de confusión:
[[1237  136]
 [ 820  214]]


In [ ]:
#Verificamos la precisión
from sklearn.metrics import precision_score
precision8 = precision_score(y_test8, y_pred8, average="binary", pos_label=1)
print('Precisión: del modelo', precision8)

Precisión: del modelo 0.6114285714285714


In [ ]:
#calculamos la exactitud del modelo
from sklearn.metrics import accuracy_score
exactitud8 = accuracy_score(y_test8, y_pred8)
print('Exactitud del modelo:', exactitud8)

Exactitud del modelo: 0.6028250934773577


In [ ]:
#Calculamos la sensibilidad del modelo
from sklearn.metrics import recall_score
sensibilidad8 = recall_score(y_test8, y_pred8, average="binary", pos_label=1)
print('Sensibilidad del modelo:', sensibilidad8)

Sensibilidad del modelo: 0.20696324951644102


**VAR 9**

In [ ]:
#Verificamos los datos unicos de la columna 'property_type'
property_unico = np.unique(df['property_type'])
property_unico 

array(['Camper/RV', 'Entire bungalow', 'Entire cabin', 'Entire chalet',
       'Entire condo', 'Entire guest suite', 'Entire guesthouse',
       'Entire home', 'Entire home/apt', 'Entire loft', 'Entire place',
       'Entire rental unit', 'Entire serviced apartment',
       'Entire townhouse', 'Entire vacation home', 'Entire villa', 'Hut',
       'Pension', 'Private room', 'Private room in bed and breakfast',
       'Private room in bungalow', 'Private room in camper/rv',
       'Private room in casa particular', 'Private room in condo',
       'Private room in farm stay', 'Private room in guest suite',
       'Private room in guesthouse', 'Private room in home',
       'Private room in hostel', 'Private room in loft',
       'Private room in rental unit',
       'Private room in serviced apartment',
       'Private room in shipping container', 'Private room in tent',
       'Private room in townhouse', 'Private room in vacation home',
       'Private room in villa', 'Room in aparthote

In [ ]:
#Mapeamos los valores de la columna 'property_type' a valores numericos
mapeo_property = {tipo: i for i, tipo in enumerate(property_unico)}

In [ ]:
# Aplicamos el mapeo a la columna 'property_type'
df['property_type'] = df['property_type'].map(mapeo_property)
df['property_type'].head(10)

0    11
1    11
2    11
3    11
4    27
5    30
6    30
7    48
8    50
9    30
Name: property_type, dtype: int64

In [ ]:
#Verificamos los datos unicos de la columna 
unico = np.unique(df['neighbourhood_cleansed'])
unico

array(['Allach-Untermenzing', 'Altstadt-Lehel', 'Au-Haidhausen',
       'Aubing-Lochhausen-Langwied', 'Berg am Laim', 'Bogenhausen',
       'Feldmoching-Hasenbergl', 'Hadern', 'Laim',
       'Ludwigsvorstadt-Isarvorstadt', 'Maxvorstadt',
       'Milbertshofen-Am Hart', 'Moosach', 'Neuhausen-Nymphenburg',
       'Obergiesing', 'Pasing-Obermenzing', 'Ramersdorf-Perlach',
       'Schwabing-Freimann', 'Schwabing-West', 'Schwanthalerhöhe',
       'Sendling', 'Sendling-Westpark',
       'Thalkirchen-Obersendling-Forstenried-Fürstenried-Solln',
       'Tudering-Riem', 'Untergiesing-Harlaching'], dtype=object)

In [ ]:
#Seleccionamos 5 zonas al azar para volver a la variable dicotomica
zonas = [
    'Obergiesing', 'Allach-Untermenzing', 'Sendling-Westpark', 
    'Schwabing-Freimann', 'Untergiesing-Harlaching'
]

In [ ]:
#Convertimos la variable a dicotomica
df['neighbourhood_cleansed'] = df['neighbourhood_cleansed'].isin(zonas).astype(int)
df['neighbourhood_cleansed'].head(10)

0    0
1    0
2    0
3    0
4    1
5    0
6    0
7    1
8    0
9    0
Name: neighbourhood_cleansed, dtype: int32

In [ ]:
#Declaramos las variables dependientes e independientes para la regresion logistica
var_ind9 = df[['property_type', 'minimum_nights', 'review_scores_location']]
var_dep9 = df[['neighbourhood_cleansed']]

In [ ]:
#Redifinimos las variables
X9 = var_ind9
y9 = var_dep9

In [ ]:
#Dividimos el conjunto de datos en un conjunto de entrenamiento y un conjunto de prueba
X_train9, X_test9, y_train9, y_test9 = train_test_split(X9, y9, test_size=0.3, random_state=None)

In [ ]:
#Se escalan todos los datos
escalar = StandardScaler()

In [ ]:

#Para realizar el escalamiento de las variables "X" tanto de entrenamiento como de prueba, se utiliza el método fit_transform
X_train9 = escalar.fit_transform(X_train9)
X_test9 = escalar.transform(X_test9)

In [ ]:
#Definimos el algoritmo a utilizar
from sklearn.linear_model import LogisticRegression
algoritmo9 = LogisticRegression()

In [ ]:
#Entrenamos el algoritmo
algoritmo9.fit(X_train9, y_train9)

c:\Python\Lib\site-packages\sklearn\utils\validation.py:1339: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


LogisticRegression()

In [ ]:

#Realizamos la prediccion
y_pred9 = algoritmo9.predict(X_test9)
y_pred9

array([0, 0, 0, ..., 0, 0, 0])

In [ ]:
#Verificamos la matriz de confusión
from sklearn.metrics import confusion_matrix
matriz9 = confusion_matrix(y_test9, y_pred9)
print('Matriz de confusión:')
print(matriz9)

Matriz de confusión:
[[2037    0]
 [ 370    0]]


In [ ]:
#Verificamos la precisión
from sklearn.metrics import precision_score
precision9 = precision_score(y_test9, y_pred9, average="binary", pos_label=1)
print('Precisión: del modelo', precision9)

Precisión: del modelo 0.0


c:\Python\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [ ]:
#calculamos la exactitud del modelo
from sklearn.metrics import accuracy_score
exactitud9 = accuracy_score(y_test9, y_pred9)
print('Exactitud del modelo:', exactitud9)

Exactitud del modelo: 0.8462816784378895


In [ ]:
#Calculamos la sensibilidad del modelo
from sklearn.metrics import recall_score
sensibilidad9 = recall_score(y_test9, y_pred9, average="binary", pos_label=0)
print('Sensibilidad del modelo:', sensibilidad9)

Sensibilidad del modelo: 1.0


**VAR 10**

In [ ]:
host_unico = np.unique(df['host_response_time'])
host_unico

array(['Estado de manera desconocida', 'a few days or more',
       'within a day', 'within a few hours', 'within an hour'],
      dtype=object)

In [ ]:
#volver a convertir la variable a dicotomica
df['host_response_time'] = df['host_response_time'].replace(['Estado de manera desconocida', 'a few days or more',
    'within a day', 'within a few hours'], 'Respond at another time')

In [ ]:
#Declaramos las variables dependientes e independientes para la regresion logistica
var_ind10 = df[['host_response_rate', 'review_scores_communication', 'host_acceptance_rate']]
var_dep10 = df[['host_response_time']]

In [ ]:
#Redifinimos las variables
X10 = var_ind10
y10 = var_dep10

In [ ]:
#Dividimos el conjunto de datos en un conjunto de entrenamiento y un conjunto de prueba
X_train10, X_test10, y_train10, y_test10 = train_test_split(X10, y10, test_size=0.3, random_state=None)

In [ ]:
#Se escalan todos los datos
escalar = StandardScaler()

In [ ]:
#Para realizar el escalamiento de las variables "X" tanto de entrenamiento como de prueba, se utiliza el método fit_transform
X_train10 = escalar.fit_transform(X_train10)
X_test10 = escalar.transform(X_test10)

In [ ]:
#Definimos el algoritmo a utilizar
from sklearn.linear_model import LogisticRegression
algoritmo10 = LogisticRegression()

In [ ]:
#Entrenamos el algoritmo
algoritmo10.fit(X_train10, y_train10)

c:\Python\Lib\site-packages\sklearn\utils\validation.py:1339: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


LogisticRegression()

In [ ]:
#Realizamos la prediccion
y_pred10 = algoritmo10.predict(X_test10)
y_pred10

array(['Respond at another time', 'Respond at another time',
       'Respond at another time', ..., 'Respond at another time',
       'Respond at another time', 'within an hour'], dtype=object)

In [ ]:
#Verificamos la matriz de confusión
from sklearn.metrics import confusion_matrix
matriz10 = confusion_matrix(y_test10, y_pred10)
print('Matriz de confusión:')
print(matriz10)

Matriz de confusión:
[[1205  371]
 [ 342  489]]


In [ ]:
#Verificamos la precisión
from sklearn.metrics import precision_score
precision10 = precision_score(y_test10, y_pred10, average="binary", pos_label="within an hour")
print('Precisión: del modelo', precision10)

Precisión: del modelo 0.5686046511627907


In [ ]:
#calculamos la exactitud del modelo
from sklearn.metrics import accuracy_score
exactitud10 = accuracy_score(y_test10, y_pred10)
print('Exactitud del modelo:', exactitud10)

Exactitud del modelo: 0.7037806398005816


In [ ]:
#Calculamos la sensibilidad del modelo
from sklearn.metrics import recall_score
sensibilidad10 = recall_score(y_test10, y_pred10, average="binary", pos_label="Respond at another time")
print('Sensibilidad del modelo:', sensibilidad10)

Sensibilidad del modelo: 0.7645939086294417
